In [77]:
from bs4 import BeautifulSoup
import requests
import re

In [90]:

url =  "https://parker-house.com/collections/tv-consoles/products/asc-90-2-dch-set"

In [91]:
# req = requests.get("https://parker-house.com/collections/tv-consoles?page=1")
req = requests.get(url)
# req = requests.get("https://parker-house.com/")

In [20]:
req.status_code

200

In [92]:

soup = BeautifulSoup(req.content, "html.parser")

In [93]:


sku = None
try:
    sku_element = soup.select_one('p.sku span[itemprop="sku"]')
    if sku_element:
        sku = sku_element.text.strip()
    if not sku:
        sku_p_tag = None
        for p_tag in soup.find_all("p"):
            if "SKU:" in p_tag.get_text():
                sku_p_tag = p_tag
                break
        if sku_p_tag:
            cleaned_text = re.sub(r"<[^>]*>", " ", str(sku_p_tag))
            cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
            match = re.search(r"SKU:\s*(\S+)", cleaned_text, flags=re.IGNORECASE)
            if match:
                sku = match.group(1).strip()
    if not sku or sku == "**":
        try:
            sku = url.split("/")[-1].upper().replace("-", "#", 1)
        except AttributeError:
            sku = ""
    if sku:
        print("The Product SKU is:", sku)
    else:
        print("SKU not found.")
except Exception as e:
    print("An error occurred while extracting SKU:", str(e))


The Product SKU is: ASC#90-2-DCH-SET


In [36]:
soup.title.text.strip()

'TV Consoles Page 2 - Parker House Furniture'

In [31]:
nav = soup.find("div", class_ = "nav")

In [53]:

direct_ul_children = nav.find_all("ul", recursive=False)


In [55]:
direct_li_children = direct_ul_children[1].find_all("li", recursive=False)


In [56]:
len(direct_li_children)

6

In [63]:
direct_li_children[5]

<li><a class="top-link" href="/collections">Collections</a></li>

In [64]:
for category in direct_li_children:
    category_name  = category.find("a").text.strip()
    print(category_name)
    if category_name != "Collections":
        


Living Room
Upholstery
Dining
Office & Library
Bedroom
Collections


## Web pages html Code saving for selectors checking

In [8]:
# Saving the prettified HTML to a file
with open("product-3.html", "w", encoding="utf-8") as file:
    file.write(soup.prettify())

## Nav Bar links scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_navbar_to_csv():
        url = "https://parker-house.com/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        nav = soup.find("div", id="nav")
        direct_ul_children = nav.find("ul")
        direct_li_children = direct_ul_children.find_all("li", recursive=False)
        csv_file = "utilities/category-collection.csv"
        with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=["category_name", "subcategory", "collection_name", "collection_link"])
            writer.writeheader()
            for category_li in direct_li_children:
                category_name = category_li.find("a").text.strip()
                print(f"Category: {category_name}")
                subcategories_ul = category_li.find("ul")
                if subcategories_ul:
                    categor = subcategories_ul.find_all("li", recursive=False)
                    for subcategory_li in categor:
                        subcategory_name = subcategory_li.find("a").text.strip()
                        
                        print(f"  Subcategory: {subcategory_name}")
                        collections_ul = subcategory_li.find("ul")
                        if collections_ul:
                            for collection_li in collections_ul.find_all("li", recursive=False):
                                collection_name = collection_li.find("a").text.strip()
                                collection_link = collection_li.find("a")["href"]
                                print(f"    Collection: {collection_name}")
                                if "collections" in subcategory_link and subcategory_name !=   "Collections":
                                    writer.writerow({
                                        "category_name": category_name,
                                        "subcategory": subcategory_name,
                                        "collection_name": collection_name.replace("-", ''),
                                        "collection_link": "https://parker-house.com" + collection_link
                                    })
                        else:
                            subcategory_link = subcategory_li.find("a")["href"]

                            if "collections" in subcategory_link and subcategory_name !=   "Collections":
                                writer.writerow({
                                    "category_name": category_name,
                                    "subcategory": subcategory_name,
                                    "collection_name": subcategory_name.replace("-", ''),
                                    "collection_link": "https://parker-house.com" + subcategory_link
                                })
                else:
                    category_link = category_li.find("a").get("href")
                    if "collections" in subcategory_link and subcategory_name !=   "Collections":
                        writer.writerow({
                            "category_name": category_name,
                            "subcategory": category_name,
                            "collection_name": category_name.replace("-", ''),
                            "collection_link": "https://parker-house.com" + category_link
                        })

        print(f"Data has been saved to {csv_file}")
        

In [ ]:
scrape_navbar_to_csv()


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_navbar_to_csv():
    url = "https://parker-house.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Locate the navigation div
    nav = soup.find("div", id="nav")
    direct_ul_children = nav.find("ul")
    direct_li_children = direct_ul_children.find_all("li", recursive=False)
    
    # Initialize the CSV file
    csv_file = "utilities/category-collection.csv"
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        # Initialize CSV writer with headers
        writer = csv.DictWriter(file, fieldnames=["category_name", "subcategory", "collection_name", "collection_link"])
        writer.writeheader()

        # Iterate through top-level categories
        for category_li in direct_li_children:
            category_name = category_li.find("a").text.strip()
            print(f"Category: {category_name}")

            # Check for subcategories
            subcategories_ul = category_li.find("ul")
            # print(subcategories_ul)
            # print("*"*30)
            # break
            if subcategories_ul:
                categor = subcategories_ul.find_all("li", recursive=False)
                for subcategory_li in categor:
                    subcategory_name = subcategory_li.find("a").text.strip()
                    
                    print(f"  Subcategory: {subcategory_name}")

                    # Check for collections under subcategories
                    collections_ul = subcategory_li.find("ul")
                    if collections_ul:
                        for collection_li in collections_ul.find_all("li", recursive=False):
                            collection_name = collection_li.find("a").text.strip()
                            collection_link = collection_li.find("a")["href"]
                            print(f"    Collection: {collection_name}")
                            if "collections" in subcategory_link and subcategory_name !=   "Collections":
                                writer.writerow({
                                    "category_name": category_name,
                                    "subcategory": subcategory_name,
                                    "collection_name": collection_name.replace("-", ''),
                                    "collection_link": "https://parker-house.com" + collection_link
                                })
                    else:
                        # No collections, treat subcategory as collection
                        subcategory_link = subcategory_li.find("a")["href"]

                        if "collections" in subcategory_link and subcategory_name !=   "Collections":
                            writer.writerow({
                                "category_name": category_name,
                                "subcategory": subcategory_name,
                                "collection_name": subcategory_name.replace("-", ''),
                                "collection_link": "https://parker-house.com" + subcategory_link
                            })
            else:
                # No subcategories, treat category as both subcategory and collection
                category_link = category_li.find("a").get("href")
                if "collections" in subcategory_link and subcategory_name !=   "Collections":
                    writer.writerow({
                        "category_name": category_name,
                        "subcategory": category_name,
                        "collection_name": category_name.replace("-", ''),
                        "collection_link": "https://parker-house.com" + category_link
                    })

    print(f"Data has been saved to {csv_file}")

# 
scrape_navbar_to_csv()


Category: Living Room
  Subcategory: Occasional Tables
  Subcategory: Entertainment
    Collection: - TV Consoles
    Collection: - Entertainment Walls
  Subcategory: Accents
    Collection: -  Bookcases
    Collection: -  Chairs
    Collection: -  Consoles
    Collection: -  Mirrors
    Collection: - Pillows
    Collection: -  Tables
Category: Upholstery
  Subcategory: Stationary
    Collection: -Sofa Groups
    Collection: -Modular Groups & Sectionals
    Collection: -Accent Chairs
    Collection: -Ottomans
    Collection: -Pillows
    Collection: -Swivel Chairs
    Collection: -Swivel Chairs w/ Ottomans
  Subcategory: Motion
  Subcategory: ‎ Leather Reclining
    Collection: -Recliners
    Collection: -Sofas
    Collection: -Loveseats
    Collection: -Modular Groups
  Subcategory: Fabric Reclining
    Collection: -Recliners
    Collection: -Sofas
    Collection: -Loveseats
    Collection: -Modular Groups
  Subcategory: Cordless Recliners
  Subcategory: Lift Chairs
  Subcategory: Zer

## Products list gathering stage

In [42]:
# Find all <a> tags
links = soup.find_all('a')

products = []

# Iterate through each link
for link in links:
    href = link.get("href")  # Get the href attribute
    if href and href.startswith("/collections/") and "/products/" in href:
        href = "https://parker-house.com" + href
        products.append(href)
        print(href)

https://parker-house.com/collections/tv-consoles/products/ame-63-cot
https://parker-house.com/collections/tv-consoles/products/ame-76-cot
https://parker-house.com/collections/tv-consoles/products/ame-92-cot
https://parker-house.com/collections/tv-consoles/products/ame-63-den
https://parker-house.com/collections/tv-consoles/products/ame-76-den
https://parker-house.com/collections/tv-consoles/products/ame-92-den
https://parker-house.com/collections/tv-consoles/products/ame-63-dov
https://parker-house.com/collections/tv-consoles/products/ame-76-dov
https://parker-house.com/collections/tv-consoles/products/ame-92-dov
https://parker-house.com/collections/tv-consoles/products/asc-90-dch
https://parker-house.com/collections/tv-consoles/products/asc-90-2-dch-set
https://parker-house.com/collections/tv-consoles/products/bel-705
https://parker-house.com/collections/tv-consoles/products/bbg-72
https://parker-house.com/collections/tv-consoles/products/boc-411
https://parker-house.com/collections/t

In [43]:
len(products)

50

In [32]:
from bs4 import BeautifulSoup
import requests

def scrape_all_products(base_url):
    current_page = 1
    products = []

    while True:
        # Request the current page
        url = f"{base_url}?page={current_page}"
        print(f"Scraping: {url}")
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")

        # Find all product links on the current page
        links = soup.find_all('a')
        for link in links:
            href = link.get("href")  # Get the href attribute
            if href and href.startswith("/collections/") and "/products/" in href:
                full_url = "https://parker-house.com" + href
                if full_url not in products:  # Avoid duplicates
                    products.append(full_url)
                    print(full_url)

        # Check for the "Next" page link
        next_page = soup.find("span", class_="next")
        if next_page and next_page.find("a"):
            current_page += 1  # Increment to the next page
        else:
            break  # No more pages, exit the loop

    return products

# Base URL for the collection
base_url = "https://parker-house.com/collections/tv-consoles"
all_products = scrape_all_products(base_url)

# Print all collected products
print("\nTotal Products Found:", len(all_products))


Scraping: https://parker-house.com/collections/tv-consoles?page=1
https://parker-house.com/collections/tv-consoles/products/ame-63-cot
https://parker-house.com/collections/tv-consoles/products/ame-76-cot
https://parker-house.com/collections/tv-consoles/products/ame-92-cot
https://parker-house.com/collections/tv-consoles/products/ame-63-den
https://parker-house.com/collections/tv-consoles/products/ame-76-den
https://parker-house.com/collections/tv-consoles/products/ame-92-den
https://parker-house.com/collections/tv-consoles/products/ame-63-dov
https://parker-house.com/collections/tv-consoles/products/ame-76-dov
https://parker-house.com/collections/tv-consoles/products/ame-92-dov
https://parker-house.com/collections/tv-consoles/products/asc-90-dch
https://parker-house.com/collections/tv-consoles/products/asc-90-2-dch-set
https://parker-house.com/collections/tv-consoles/products/bel-705
https://parker-house.com/collections/tv-consoles/products/bbg-72
https://parker-house.com/collections/t

In [51]:
import csv
import requests
from bs4 import BeautifulSoup
import os

def scrape_all_products(base_url):
    current_page = 1
    products = []

    while True:
        # Request the current page
        url = f"{base_url}?page={current_page}"
        print(f"Scraping: {url}")
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")

        # Find all product links on the current page
        links = soup.find_all('a')
        for link in links:
            href = link.get("href")  # Get the href attribute
            if href and href.startswith("/collections/") and "/products/" in href:
                full_url = "https://parker-house.com" + href
                if full_url not in products:  # Avoid duplicates
                    products.append(full_url)
                    print(full_url)

        # Check for the "Next" page link
        next_page = soup.find("span", class_="next")
        if next_page and next_page.find("a"):
            current_page += 1  # Increment to the next page
        else:
            break  # No more pages, exit the loop

    return products

def process_collection_products(collection, all_products):
    category_name = collection['category_name']
    subcategory_name = collection["subcategory"]
    collection_name = collection['collection_name']
    collection_link = collection['collection_link']

    print(f"Processing category: {category_name}, collection: {collection_name}")

    # Scrape all products for the current collection
    product_links = scrape_all_products(collection_link)
    print(f"Products found: {len(product_links)}")

    # Add product data to the list
    for link in product_links:
        all_products.append({
            'category_name': category_name,
            'subcategory_name': subcategory_name,
            'collection_name': collection_name,
            'product_link': link
        })

def get_collections_products():
    # Ensure directories exist
    input_dir = 'utilities'
    os.makedirs(input_dir, exist_ok=True)


    input_file = os.path.join(input_dir, 'category-collection.csv')
    output_file = os.path.join(input_dir, 'products-links.csv')

    # Check if the input file exists; if not, create a sample file
    if not os.path.exists(input_file):
        print(f"{input_file} not found. Creating a sample file...")
        with open(input_file, 'w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['category_name', 'subcategory', 'collection_name', 'collection_link'])
            writer.writeheader()
            writer.writerow({
                'category_name': 'Sample Category',
                'subcategory': 'Sample Subcategory',
                'collection_name': 'Sample Collection',
                'collection_link': 'https://example.com/collections/sample-collection'
            })
        print(f"Sample file created at {input_file}. Please update it with real data and rerun the script.")
        return

    all_products = []  # Initialize the list to hold all product data

    # Read the input CSV file
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        collections = [row for row in reader]

    # Process each collection
    for collection in collections:
        process_collection_products(collection, all_products)

    # Write the output to a CSV file
    with open(output_file, 'w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['category_name', 'subcategory_name', 'collection_name', 'product_link'])
        writer.writeheader()
        writer.writerows(all_products)

    print("Scraping completed. Results saved to", output_file)

# Run the scraping process
get_collections_products()


Processing category: Living Room, collection: Occasional Tables
Scraping: https://parker-house.com/collections/occasional-tables?page=1
https://parker-house.com/collections/occasional-tables/products/ame-06-cot
https://parker-house.com/collections/occasional-tables/products/ame-01-cot
https://parker-house.com/collections/occasional-tables/products/ame-05-cot
https://parker-house.com/collections/occasional-tables/products/ame-02-cot
https://parker-house.com/collections/occasional-tables/products/ame-09-cot
https://parker-house.com/collections/occasional-tables/products/ame-09-4-cot
https://parker-house.com/collections/occasional-tables/products/ame-06-den
https://parker-house.com/collections/occasional-tables/products/ame-01-den
https://parker-house.com/collections/occasional-tables/products/ame-02-den
https://parker-house.com/collections/occasional-tables/products/ame-06-dov
https://parker-house.com/collections/occasional-tables/products/ame-01-dov
https://parker-house.com/collections/

## Product page scraping

In [8]:
product_title = soup.find("h1", class_ = "product_name").text.strip()

In [9]:
product_title

'AMERICANA MODERN BEDROOM 3 Drawer Nightstand with charging station'

In [10]:
sku = soup.find("p", class_ = "sku").text.strip()

In [11]:
sku

'BAME#51303-COT'

In [12]:
description = soup.find("div", id = "tabs-3")
if description:
    description = description.find("p").text.strip()

In [13]:
description

'This three-drawer nightstand is a charming and functional piece of furniture that brings timeless style to your bedroom. The fresh hand-painted Cotton finish with contrasting Weathered Natural top is accented by the solid-metal custom hardware in authentic Burnished Bronze, creating a warm and inviting atmosphere. Crafted from the finest materials, including poplar solids, birch veneers, and oak veneer tops, this nightstand is designed to last for years to come.'

In [16]:
products_img = soup.find_all("a", class_ = "fancybox")

if products_img:
    products_images = list(set(["https:" + item.get("href") for item in products_img]))

In [17]:
products_images

['https://parker-house.com/cdn/shop/products/AME_51303-COT_top_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AME_51303-COT_back_angle_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AME_51303-COT_back_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AME_51303-COT_side_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/files/AME_51303-COT--cutout_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AME_51303-COT_front_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AME_51303-COT_angle_silo_2048x.jpg?v=1685113049',
 'https://parker-house.com/cdn/shop/products/AMERICANA_Bedroom_LAM_a19f2d85-b8c4-4b3b-b3c9-ef504d67eada_2048x.jpg?v=1685113049']

In [20]:
# Extract data
sku = soup.find("p", class_ = "sku").text.strip()

product_title = soup.find("h1", class_ = "product_name").text.strip()

description = soup.find("div", id = "tabs-3")
if description:
    description = description.find("p").text.strip()


products_img = soup.find_all("a", class_ = "fancybox")
if products_img:
    products_images = list(set(["https:" + item.get("href") for item in products_img]))

width = soup.find("td", text="Width").find_next_sibling("td").text.strip()

depth = soup.find("td", text="Depth").find_next_sibling("td").text.strip()

height = soup.find("td", text="Height").find_next_sibling("td").text.strip()

finish = soup.find("div", id="tabs-5").find("p").text.strip().split("Material:")[0].replace("Finish:", "").strip()

material = soup.find("div", id="tabs-5").find("p").text.strip().split("Material:")[1].strip()

features = [li.text.strip() for li in soup.find("div", id="tabs-6").find_all("li")]



/tmp/ipykernel_8693/2695367613.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  width = soup.find("td", text="Width").find_next_sibling("td").text.strip()
/tmp/ipykernel_8693/2695367613.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  depth = soup.find("td", text="Depth").find_next_sibling("td").text.strip()
/tmp/ipykernel_8693/2695367613.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  height = soup.find("td", text="Height").find_next_sibling("td").text.strip()


In [21]:
width

'30.5 in.'

In [22]:
depth

'17 in.'

In [23]:
height

'29 in.'

In [24]:
finish

'Cotton With Weathered Natural Top'

In [25]:
material

'Poplar Solids Oak & Birch Veneers'

In [26]:
features

['Three generously sized drawers make it easy to keep bedtime essentials close at hand',
 'Innovative touch-lighting adds nightlight effect to base',
 'Integrated USB/ USB-C charger keeps phones and devices powered',
 'Fresh hand-painted Cotton finish with contrasting Weathered Natural top offers timeless charm',
 'Solid-metal custom hardware in authentic Burnished Bronze',
 'Crafted of poplar solids with birch veneers and oak veneer tops',
 'Pairs beautifully with Americana Modern beds',
 'Metal ball bearing drawer glides',
 'Tapered legs create an eye-catching silhouette',
 'Select woods are kiln-dried']

In [27]:
# Extract data with try-except for each selector
try:
    sku = soup.find("p", class_="sku").text.strip()
    print("SKU : ", sku)
except AttributeError:
    sku = "N/B"

try:
    product_title = soup.find("h1", class_="product_name").text.strip()
except AttributeError:
    product_title = "N/B"

try:
    description = soup.find("div", id="tabs-3").find("p").text.strip()
except AttributeError:
    description = "N/B"

try:
    products_img = soup.find_all("a", class_="fancybox")
    products_images = list(set(["https:" + item.get("href") for item in products_img]))
except AttributeError:
    products_images = "N/B"

try:
    width = soup.find("td", text="Width").find_next_sibling("td").text.strip()
except AttributeError:
    width = "N/B"

try:
    depth = soup.find("td", text="Depth").find_next_sibling("td").text.strip()
except AttributeError:
    depth = "N/B"

try:
    height = soup.find("td", text="Height").find_next_sibling("td").text.strip()
except AttributeError:
    height = "N/B"

try:
    finish = soup.find("div", id="tabs-5").find("p").text.strip().split("Material:")[0].replace("Finish:", "").strip()
except (AttributeError, IndexError):
    finish = "N/B"

try:
    material = soup.find("div", id="tabs-5").find("p").text.strip().split("Material:")[1].strip()
except (AttributeError, IndexError):
    material = "N/B"

try:
    features = [li.text.strip() for li in soup.find("div", id="tabs-6").find_all("li")]
except AttributeError:
    features = "N/B"


SKU :  BAME#51303-COT


/tmp/ipykernel_8693/1434289279.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  width = soup.find("td", text="Width").find_next_sibling("td").text.strip()
/tmp/ipykernel_8693/1434289279.py:30: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  depth = soup.find("td", text="Depth").find_next_sibling("td").text.strip()
/tmp/ipykernel_8693/1434289279.py:35: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  height = soup.find("td", text="Height").find_next_sibling("td").text.strip()


In [6]:
link = "https://parker-house.com/collections/freemotion-included/products/mque-812gsph-p25-upmu"

sku = link.split("/")[-1].upper()

In [35]:


# Initialize variables
try:
    finish = soup.find("em", text="Finish: ").find_parent("b").next_sibling.strip()
except AttributeError:
    finish = ""

try:
    material = soup.find("em", text="Material: ").find_parent("b").next_sibling.strip()
except AttributeError:
    material = ""

# Output the results
print("Finish:", finish)
print("Material:", material)


Finish: Cotton With Weathered Natural Top
Material: Poplar Solids Oak & Birch Veneers


/tmp/ipykernel_28887/1321582189.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish = soup.find("em", text="Finish: ").find_parent("b").next_sibling.strip()
/tmp/ipykernel_28887/1321582189.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  material = soup.find("em", text="Material: ").find_parent("b").next_sibling.strip()


In [36]:

try:
    description_id = soup.find("a", text="Description")
    description = soup.find("div", id=description_id).find("p").text.strip()
except AttributeError:
    description = ""

/tmp/ipykernel_28887/2427238136.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_id = soup.find("a", text="Description")


In [43]:

try:
    description_id = soup.find("a", text="Description")
    # print(description_id)
    if description_id:
        description_id = description_id.get("href").replace("#", "")
        # print(description_id)
        description = soup.find("div", id=description_id).find("p").text.strip()
    else:
        description = ""
except AttributeError:
    description = ""

<a href="#tabs-3">Description</a>
tabs-3


/tmp/ipykernel_28887/753738385.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_id = soup.find("a", text="Description")


In [44]:
description

'This three-drawer nightstand is a charming and functional piece of furniture that brings timeless style to your bedroom. The fresh hand-painted Cotton finish with contrasting Weathered Natural top is accented by the solid-metal custom hardware in authentic Burnished Bronze, creating a warm and inviting atmosphere. Crafted from the finest materials, including poplar solids, birch veneers, and oak veneer tops, this nightstand is designed to last for years to come.'

In [45]:
try:
    features_id = soup.find("a", text="Features")
    if features_id:
        features_id = features_id.get("href").replace("#", "")
        # print(features_id)
        features = [li.text.strip() for li in soup.find("div", id=features_id).find_all("li")]
except AttributeError:
    features = ""

tabs-6


/tmp/ipykernel_28887/3498656398.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  features_id = soup.find("a", text="Features")


In [46]:
features

['Three generously sized drawers make it easy to keep bedtime essentials close at hand',
 'Innovative touch-lighting adds nightlight effect to base',
 'Integrated USB/ USB-C charger keeps phones and devices powered',
 'Fresh hand-painted Cotton finish with contrasting Weathered Natural top offers timeless charm',
 'Solid-metal custom hardware in authentic Burnished Bronze',
 'Crafted of poplar solids with birch veneers and oak veneer tops',
 'Pairs beautifully with Americana Modern beds',
 'Metal ball bearing drawer glides',
 'Tapered legs create an eye-catching silhouette',
 'Select woods are kiln-dried']

In [5]:
from bs4 import BeautifulSoup

html_content = """
<p>
  <b>
    <em>
      Cover:
    </em>
  </b>
  Soft Ivory
  <br/>
  <b>
    <em>
      Material:
    </em>
  </b>
  90% Polyester 10% PU (W)
  <br/>
  <em>
    <strong>
      SKU:&nbsp;
    </strong>
  </em>
  MGEM#812LIFT-SFIV
</p>
"""

soup = BeautifulSoup(html_content, "html.parser")

# Locate the <strong> tag with the text "SKU:"
sku_strong = soup.find("strong", text="SKU:&nbsp;")
if sku_strong:
    # Get the parent <em> tag
    sku_em = sku_strong.find_parent("em")
    if sku_em:
        # Get the next string after the <em> tag
        sku = sku_em.find_next(string=True).strip()
    else:
        sku = ""
else:
    sku = ""

print("SKU value:", sku)


SKU value: 


/tmp/ipykernel_9985/2989615887.py:31: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  sku_strong = soup.find("strong", text="SKU:&nbsp;")


In [10]:
sku = soup.find("strong", text="SKU:")

/tmp/ipykernel_9985/3195596734.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  sku = soup.find("strong", text="SKU:")


In [11]:
sku

In [16]:


# Find the element with a string containing "#"
sku_element = soup.find(string=lambda text: text and "#" in text)
if sku_element:
    sku = sku_element.strip()
else:
    sku = ""

print("SKU value:", sku)


SKU value: #dynamic-checkout-cart {
  container-type: inline-size;
  container-name: dcc;
  width: 100%;
}

.wallet-cart-grid {
  --wallet-button-height-horizontal: clamp(
    25px,
    var(
      --shopify-accelerated-checkout-button-inline-size,
      42px
    ),
    55px
  );
  --wallet-button-height-vertical: clamp(
    25px,
    var(
      --shopify-accelerated-checkout-button-block-size,
      54px
    ),
    55px
  );
  --wallet-button-width-horizontal: 150px;
  --wallet-button-width-vertical: 100%;
  --wallet-button-border-radius: var(
    --shopify-accelerated-checkout-button-border-radius,
    4px
  );
  --wallet-grid-margin-horizontal: 0 -5px -5px -5px;
  --wallet-button-container-margin-horizontal: 0 5px 5px;
  --wallet-button-container-margin-vertical: var(--shopify-accelerated-checkout-row-gap, 8px) 0 0;
}

@keyframes acceleratedCheckoutLoadingSkeleton {
  50% {opacity: var(--shopify-accelerated-checkout-skeleton-animation-opacity-start, 1);}
  75% {opacity: var(--shopify

In [17]:
from bs4 import BeautifulSoup

html_content = """
<p>
  <b>
    <em>
      Cover:
    </em>
  </b>
  Soft Ivory
  <br/>
  <b>
    <em>
      Material:
    </em>
  </b>
  90% Polyester 10% PU (W)
  <br/>
  <em>
    <strong>
      SKU:&nbsp;
    </strong>
  </em>
  MGEM#812LIFT-SFIV
</p>
"""

soup1 = BeautifulSoup(html_content, "html.parser")

# Find the element with a string containing "#" only in <p> tags
sku_element = soup1.find(string=lambda text: text and "#" in text and text.find_parent("p"))
if sku_element:
    sku = sku_element.strip()
else:
    sku = ""

print("SKU value:", sku)


SKU value: MGEM#812LIFT-SFIV


In [25]:
# Find the element with a string containing "#" only in <p> tags
sku_element = soup1.find(string=lambda text: text and "#" in text and text.find_parent("p"))
if sku_element:
    sku = sku_element.strip()
else:
    sku = ""

print("SKU value:", sku)

SKU value: MGEM#812LIFT-SFIV


In [30]:
sku = ""

# Find all <p> tags
for p_tag in soup.find_all("p"):
    # Check for a string containing "#"
    sku_element = p_tag.find(string=lambda text: text and "#" in text)
    if sku_element:
        sku = sku_element.strip()
        break  # Stop once the first match is found

    # Check for <span> inside <p> containing "#"
    span_element = p_tag.find("span", string=lambda text: text and "#" in text)
    if span_element:
        sku = span_element.text.strip()
        break  # Stop once the first match is found

print("SKU value:", sku)

SKU value: 


In [36]:
sku = next(
        (text.strip() for p in soup.find_all("p") for text in p.stripped_strings if "#" in text),
        None
    )

if sku:
    print("SKU value:", sku)
else:
    print("No SKU found on the page.")

No SKU found on the page.


In [37]:
sku = next(
        (text.strip() for p in soup.find_all("p") for text in p.stripped_strings if "**" in text),
        None
    )

if sku:
    print("SKU value:", sku)
else:
    print("No SKU found on the page.")

SKU value: **MPRM-PACKB(H)-MDE SET


In [38]:
import re


In [39]:
pattern = r"[A-Z\-()0-9]+"

# Find the first <p> tag that contains a string matching the SKU pattern
sku = next(
    (text.strip() for p in soup.find_all("p") for text in p.stripped_strings if re.search(pattern, text)),
    None
)

if sku:
    print("SKU value:", sku)
else:
    print("No SKU found on the page.")

SKU value: SKU:


In [17]:
try:
    color_id = soup.find("a", text ='Color')
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id =color_id)
    
except AttributeError:
    color = ""

/tmp/ipykernel_14129/3433690609.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text ='Color')


In [18]:
color.

<div id="tabs-4">
<p><b><em>Cover: </em></b>Magic Denim<br/><b><em>Material: </em></b>100% Polyester (W)<br/><br/></p>
</div>

In [23]:
material = ""
finish =""

In [24]:
if color:
    # Extract Finish
    finish_em = color.find("em", text="Finish:")
    if finish_em:
        finish_b = finish_em.find_parent("b")
        if finish_b:
            finish = finish_b.next_sibling.strip() if finish_b.next_sibling else ""

    # Extract Material
    material_em = color.find("em", text="Material:")
    if material_em:
        material_b = material_em.find_parent("b")
        if material_b:
            
            material_span = material_b.find_next("span")
            if material_span:
                material = material_span.text.strip()
            else:
                material = material_b.next_sibling.strip() if material_b.next_sibling else ""


/tmp/ipykernel_14129/3427521897.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_em = color.find("em", text="Finish:")
/tmp/ipykernel_14129/3427521897.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  material_em = color.find("em", text="Material:")


In [25]:
material

''

In [33]:
try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

material = ""
finish = ""

if color:
    # Extract Finish
    try:
        finish_em = color.find("em", text="Finish:")
        if finish_em:
            finish_b = finish_em.find_parent("b")
            if finish_b:
                finish = finish_b.next_sibling.strip() if finish_b.next_sibling else ""
    except AttributeError:
        finish = ""

    # Extract Material
    try:
        material_em = color.find("em", text="Material:")
        if material_em:
            material_b = material_em.find_parent("b")
            if material_b:
                # Check for span or sibling text
                material_span = material_b.find_next("span")
                if material_span:
                    material = material_span.text.strip()
                else:
                    material = material_b.next_sibling.strip() if material_b.next_sibling else ""
    except AttributeError:
        material = ""

# Print results for verification
print("Finish:", finish)
print("Material:", material)


Finish: 
Material: 


/tmp/ipykernel_14129/1052212269.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")
/tmp/ipykernel_14129/1052212269.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_em = color.find("em", text="Finish:")
/tmp/ipykernel_14129/1052212269.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  material_em = color.find("em", text="Material:")


In [41]:
try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

/tmp/ipykernel_14129/3374418741.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")


In [42]:
color

<div id="tabs-4">
<p><b><em>Cover: </em></b>Magic Denim<br/><b><em>Material: </em></b>100% Polyester (W)<br/><br/></p>
</div>

In [43]:
try:
    finish = color.find("em", text="Finish: ").find_parent("b").next_sibling.strip()
    print("Finish : ", finish)
except (AttributeError, IndexError):
    finish = ""

try:
    material_em = color.find("em", text="Material: ")
    if material_em:
        material_b = material_em.find_parent("b")
        if material_b:
            material_span = material_b.find_next("span")
            if material_span:
                material = material_span.text.strip()
            else:
                material = material_b.next_sibling.strip()
        else:
            material = ""
    else:
        material = ""
    print("Material:", material)
except (AttributeError, IndexError):
    material = ""

Material: Add to Cart


/tmp/ipykernel_14129/303116303.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish = color.find("em", text="Finish: ").find_parent("b").next_sibling.strip()
/tmp/ipykernel_14129/303116303.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  material_em = color.find("em", text="Material: ")


In [44]:
finish

''

In [47]:
color = color.find("p")

In [48]:
color

<p><b><em>Cover: </em></b>Magic Denim<br/><b><em>Material: </em></b>100% Polyester (W)<br/><br/></p>

In [50]:
color.get_text()

'Cover:\xa0Magic DenimMaterial: 100% Polyester (W)'

In [49]:
try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

# Extract the first <p> tag within the `color` div
if color:
    color = color.find("p")

    color.replace("</em>", " ").replace("<em>", " ").replce("<br/>", " ").replace("")

# Initialize variables
finish = ""
material = ""

if color:
    # Extract Finish
    finish_em = color.find("em", text="Finish:")
    if finish_em:
        finish_b = finish_em.find_parent("b")
        if finish_b:
            finish = finish_b.next_sibling.strip() if finish_b.next_sibling else ""

    # Extract Material
    material_em = color.find("em", text="Material:")
    if material_em:
        material_b = material_em.find_parent("b")
        if material_b:
            material = material_b.next_sibling.strip() if material_b.next_sibling else ""

# Print results
print("Finish:", finish)
print("Material:", material)


Finish: 
Material: 


/tmp/ipykernel_14129/647401977.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")
/tmp/ipykernel_14129/647401977.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  finish_em = color.find("em", text="Finish:")
/tmp/ipykernel_14129/647401977.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  material_em = color.find("em", text="Material:")


In [ ]:

 #Checks if the sku is rendered the normal way, and gets its value 
if response.css('p.sku span') :
sku = response.css('p.sku span::text').get()
print('The Product SKU is: ', sku)

#Searches for a <p> tag that has the text 'SKU:' in it then extracts the value of the SKU
else:
tabs_text = response.css('div#tabs div p').getall()
sku_p_tag = None
for p_tag in tabs_text:
    if 'SKU:' in p_tag:
        sku_p_tag = p_tag
        break
if sku_p_tag:
    sku_cleaned = sku_p_tag.replace('<p>', '').replace('</p>', '') \
                    .replace('<b>', '').replace('</b>', '') \
                    .replace('<em>', '').replace('</em>', '') \
                    .replace('<span>', '').replace('</span>', '') \
                    .replace('<strong>', '').replace('</strong>', '') \
                    .strip()
    if '<br>' in sku_cleaned:
        sku_cleaned = sku_cleaned.split('<br>')[-1].strip()

    sku = sku_cleaned.replace('SKU:', '').strip()
    print('The Product SKU is: ', sku)

In [ ]:

 #Checks if the sku is rendered the normal way, and gets its value 
if response.css('p.sku span') :
sku = response.css('p.sku span::text').get()
print('The Product SKU is: ', sku)

#Searches for a <p> tag that has the text 'SKU:' in it then extracts the value of the SKU
else:
tabs_text = response.css('div#tabs div p').getall()
sku_p_tag = None
for p_tag in tabs_text:
    if 'SKU:' in p_tag:
        sku_p_tag = p_tag
        break
if sku_p_tag:
    sku_cleaned = sku_p_tag.replace('<p>', '').replace('</p>', '') \
                    .replace('<b>', '').replace('</b>', '') \
                    .replace('<em>', '').replace('</em>', '') \
                    .replace('<span>', '').replace('</span>', '') \
                    .replace('<strong>', '').replace('</strong>', '') \
                    .strip()
    if '<br>' in sku_cleaned:
        sku_cleaned = sku_cleaned.split('<br>')[-1].strip()

    sku = sku_cleaned.replace('SKU:', '').strip()
    print('The Product SKU is: ', sku)

In [ ]:

 #Checks if the sku is rendered the normal way, and gets its value 
if response.css('p.sku span') :
sku = response.css('p.sku span::text').get()
print('The Product SKU is: ', sku)

#Searches for a <p> tag that has the text 'SKU:' in it then extracts the value of the SKU
else:
tabs_text = response.css('div#tabs div p').getall()
sku_p_tag = None
for p_tag in tabs_text:
    if 'SKU:' in p_tag:
        sku_p_tag = p_tag
        break
if sku_p_tag:
    sku_cleaned = sku_p_tag.replace('<p>', '').replace('</p>', '') \
                    .replace('<b>', '').replace('</b>', '') \
                    .replace('<em>', '').replace('</em>', '') \
                    .replace('<span>', '').replace('</span>', '') \
                    .replace('<strong>', '').replace('</strong>', '') \
                    .strip()
    if '<br>' in sku_cleaned:
        sku_cleaned = sku_cleaned.split('<br>')[-1].strip()

    sku = sku_cleaned.replace('SKU:', '').strip()
    print('The Product SKU is: ', sku)

In [ ]:
<p><b><em>Cover: </em></b>Magic Denim<br/><b><em>Material: </em></b>100% Polyester (W)<br/><br/></p>


In [55]:
try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

# Initialize variables
finish = ""
material = ""

# Extract the first <p> tag and clean its content
if color:
    p_tag = color.find("p")
    if p_tag:
        # Clean the HTML content
        raw_text = str(p_tag).replace("</em>", " ").replace("<em>", " ").replace("<br/>", " ").replace("<br>", " ").replace("<b>", "").replace("</b>", "").replace("<span>", "").replace("</span>", "").strip()

        # Extract Finish
        if "Finish:" in raw_text:
            finish_start = raw_text.find("Finish:") + len("Finish:")
            finish_end = raw_text.find("Material:") if "Material:" in raw_text else len(raw_text)
            finish = raw_text[finish_start:finish_end].strip()

        # Extract Material
        if "Material:" in raw_text:
            material_start = raw_text.find("Material:") + len("Material:")
            material_end = raw_text.find("Also Available In:") if "Also Available In:" in raw_text else len(raw_text)
            material = raw_text[material_start:material_end].strip()

# Print results
print("Finish:", finish)
print("Material:", material)


Finish: Light Washed Natural
Material: <span data-mce-fragment="1">Solid Mango with Bone Inlay  </p>


/tmp/ipykernel_14129/851684526.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")


In [56]:
import re

try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

# Initialize variables
finish = ""
material = ""

# Extract the first <p> tag and clean its content
if color:
    p_tag = color.find("p")
    if p_tag:
        # Convert the <p> tag to a string
        raw_text = str(p_tag)
        
        # Remove all HTML tags and their attributes using regex
        clean_text = re.sub(r"<[^>]*>", " ", raw_text)
        clean_text = re.sub(r"\s+", " ", clean_text).strip()  # Normalize whitespace

        # Extract Finish
        if "Finish:" in clean_text:
            finish_start = clean_text.find("Finish:") + len("Finish:")
            finish_end = clean_text.find("Material:") if "Material:" in clean_text else len(clean_text)
            finish = clean_text[finish_start:finish_end].strip()

        # Extract Material
        if "Material:" in clean_text:
            material_start = clean_text.find("Material:") + len("Material:")
            material_end = clean_text.find("Also Available In:") if "Also Available In:" in clean_text else len(clean_text)
            material = clean_text[material_start:material_end].strip()

# Print results
print("Finish:", finish)
print("Material:", material)


Finish: Light Washed Natural
Material: Solid Mango with Bone Inlay


/tmp/ipykernel_14129/1716176131.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")


In [71]:
import re

try:
    color_id = soup.find("a", text="Color")
    if color_id:
        color_id = color_id.get("href").replace("#", "")
        color = soup.find("div", id=color_id)
    else:
        color = None
except AttributeError:
    color = None

# Initialize variables
finish = ""
material = ""

# Extract the first <p> tag and clean its content
if color:
    p_tag = color.find("p")
    if p_tag:
        # Convert the <p> tag to a string
        raw_text = str(p_tag)
        
        # Remove all HTML tags and their attributes using regex
        clean_text = re.sub(r"<[^>]*>", " ", raw_text)
        
        # Remove SKU and its associated content
        clean_text = re.sub(r"SKU:\s*\S+", " ", clean_text, flags=re.IGNORECASE)
        
        # Normalize whitespace
        clean_text = re.sub(r"\s+", " ", clean_text).strip()

        # Extract Finish
        if "Finish:" in clean_text:
            finish_start = clean_text.find("Finish:") + len("Finish:")
            finish_end = clean_text.find("Material:") if "Material:" in clean_text else len(clean_text)
            finish = clean_text[finish_start:finish_end].strip()

        # Extract Material
        if "Material:" in clean_text:
            material_start = clean_text.find("Material:") + len("Material:")
            material_end = clean_text.find("Also Available In:") if "Also Available In:" in clean_text else len(clean_text)
            material = clean_text[material_start:material_end].strip()

# Print results
print("Finish:", finish)
print("Material:", material)


Finish: 
Material: 90% Polyester 10% PU (W)


/tmp/ipykernel_14129/767834174.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  color_id = soup.find("a", text="Color")


In [72]:
sku_p_tag = None
for p_tag in soup.find_all("p"):
    if "SKU:" in p_tag.get_text():
        sku_p_tag = p_tag
        break
if sku_p_tag:
    cleaned_text = re.sub(r"<[^>]*>", " ", str(sku_p_tag))
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    match = re.search(r"SKU:\s*(\S+)", cleaned_text, flags=re.IGNORECASE)
    if match:
        sku = match.group(1).strip()
        print('The Product SKU is:', sku)
    else:
        sku = None
        print("SKU not found.")
else:
    sku = None
    print("SKU not found in the HTML.")


The Product SKU is: MGEM#812LIFT-SFIV
